### Question 5

In [1]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

### Optimization Problem

For the training set, we simply need to find optimal $\theta$ by minimizing a convex function (sum of norms of affine functions). The problem is formulated as <br>
$min. \frac{1}{2 N} \sum_{i=1}^{N}\left\|\pi_{i}-\theta x_{i}\right\|$ <br>
There are no real constraints to this problem. Finally, we will measure the performance of our predictor on the test set using the average distance metric <br>
$\frac{1}{2 N^{\text {test }}} \sum_{i=1}^{N^{\text {test }}}\left\|\pi_{i}^{\mathrm{test}}-P\left(x_{i}^{\mathrm{test}}\right)\right\|_{1}$

### Implementation

In [2]:
def Pi(y):
    """Returns the rakning of the argument.

    :param y: a 2d array of size N x K or a 1d array of size K, N input vectors/1 vector to generate the rankings
    
    :return: a 2d numpy array with ith row being the ranking of the ith row of y if y is a 2d array, 
    and a 1d ranking (numpy array) if y is a 1d array
    """
    y = np.array(y)
    if y.ndim == 1:
        ranking = np.argsort(np.argsort(y)) + 1
    else:
        ranking = np.argsort(np.argsort(y, axis = -1), axis = -1) + 1
    return ranking

In [3]:
# Data generation

N = N_test = 500
d = 20
K = 10

np.random.seed(0)

# Generate the true theta matrix
theta_true = np.random.randn(K,d)
theta_true /= np.linalg.norm(theta_true[:])

# Sample x_i from standard Gaussian
X_test = np.hstack([np.random.randn(N_test,  d-1), np.ones((N_test, 1))])
X_train = np.hstack([np.random.randn(N,  d-1), np.ones((N, 1))])

# Generate the true features y = theta x and add noise to them
Y_train, Y_test = X_train.dot(theta_true.T), X_test.dot(theta_true.T)

# Add 15dB of noise to the observed y to generate noisy rankings
noise_snr = 15.
sigma_noise = 10 ** (-0.05 * noise_snr) / np.sqrt(K)
Y_train, Y_test = Y_train + sigma_noise * np.random.randn(N, K), Y_test + sigma_noise * np.random.randn(N_test, K)

# Get the rankins of the observed noisy data
pi_train, pi_test = Pi(Y_train), Pi(Y_test)

In [4]:
# defining variable

theta = cp.Variable((K, d))

In [5]:
# defining objective function

objf = 0

for i in range(N):
    objf += cp.norm(pi_train[i] - theta@X_train[i], 1)
    
obj = cp.Minimize((1/(2*N))*objf)

In [6]:
# solve the problem

prob = cp.Problem(obj, [])
prob.solve();

In [7]:
# calculate rankings for train and test data

pred_train = np.zeros((N, K))
for i in range(N):
    pred_train[i] = Pi(theta.value@X_train[i])

pred_test = np.zeros((N_test, K))
for i in range(N_test):
    pred_test[i] = Pi(theta.value@X_test[i])

In [8]:
# calculate distance metrics for train and test data

avg_dist_train = (1/(2*N))*(np.sum(np.linalg.norm(pi_train-pred_train, 1, axis=1)))
avg_dist_test = (1/(2*N))*(np.sum(np.linalg.norm(pi_test-pred_test, 1, axis=1)))

In [9]:
print("The average distance on the training set is {}".format(avg_dist_train))
print("The average distance on the test set is {}".format(avg_dist_test))

The average distance on the training set is 2.488
The average distance on the test set is 2.69
